In [7]:
## test utils
%reload_ext autoreload
%autoreload 2

import os
from hiseq.utils.utils import *
from hiseq.utils.file import list_dir

f = '/data/yulab/wangming/work/yu_2021/projects/20211024_qd_pol2-yn/results/CnT/M-polII'
a = read_hiseq(f)
# a.hiseq_type
# list_hiseq_dir(f, 'rx')
def qc_trim_summary(x, hiseq_type='r1'):
    """
    # format:
    # name, input, output, out_pct, rm_pct
    """
    a = read_hiseq(x, hiseq_type) # for general usage
    if hiseq_type == 'auto':
        hiseq_type = a.hiseq_type # auto
    if not a.is_hiseq:
        log.error('qc_trim_summary() skipped, not a hiseq dir: {}'.format(x))
        return None
    if a.is_hiseq_r1:
        # option-1: stat.yaml
        # option-2: stat.txt
        stat_json = getattr(a, 'trim_json', None)
        stat_txt = getattr(a, 'trim_stat', None)
        # format:
        # name, total, too_short, dup, too_short2, clean, percent
        d = {
            'name': a.smp_name,
            'input': 1,
            'output': 1,
            'out_pct': 100.0,
            'rm_pct': 0,
        }
        if file_exists(stat_json):
            df = Config().load(stat_json) # laod data
            d['input'] = int(df.get('total', 1))
            d['output'] = int(df.get('clean', 1))
            d['out_pct'] = float(df.get('percent', 100.0))
            d['rm_pct'] = 100.0 - d['out_pct']
        elif file_exists(stat_txt):
            try:
                s = None # init
                with open(stat_txt) as r:
                    for line in r:
                        if line.startswith('#'):
                            continue
                        s = line.strip().split('\t')
                        break
                if isinstance(s, list):
                    d = {
                        'name': s[0],
                        'input': int(s[1]),
                        'output': int(s[-2]),
                        'out_pct': float(s[-1]),
                        'rm_pct': 100.0 - float(s[-1]),
                    }
            except IOError as e:
                log.error(e)
        else:
            log.error('trim.stat not exists: {}'.format(stat_txt))
        # update pct
        d['out_pct'] = float('{:.2f}'.format(d['out_pct']))
        d['rm_pct'] = float('{:.2f}'.format(d['rm_pct']))
        # save to new file
        Config().dump(d, a.trim_summary_json)
    elif a.is_hiseq_rn:
        r1 = list_hiseq_file(x, 'trim_summary_json', 'r1')
        d = {}
        for i in r1:
            di = Config().load(i)
            d = {k:d.get(k, 0)+di.get(k, 0) for k,v in di.items() if type(v) == int} # merge values
            d_str = {k:v for k,v in di.items() if type(v) == bool or type(v) == str}
            d.update(d_str) # unique_only, index, name
            d['name'] = list_hiseq_file(x, 'smp_name', 'auto') # update name
            # update out_pct, rm_pct
            d['out_pct'] = round(d.get('output', 0)/d.get('input', 1)*100, 2)
            d['rm_pct'] = round(100-d.get('out_pct'), 2)
        Config().dump(d, a.trim_summary_json)
    else:
        log.warning('qc_align_summary() skipped, no align_json')
        return None
    return d



f = '/data/yulab/wangming/work/yu_2021/projects/20211024_qd_pol2-yn/results/CnT/M-polII'
# qc_align_summary(f, 'rn')
qc_trim_summary(f, 'rn')
# list_hiseq_file(f, 'smp_name', 'rn')

{'input': 2, 'output': 2, 'name': 'M-polII', 'out_pct': 100.0, 'rm_pct': 0.0}

In [ ]:
## test utils
%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from hiseq.utils.file import list_file
from utils import *
from hiseq.utils.utils import (
    log, update_obj, Config, get_date, read_hiseq, list_hiseq_file, list_hiseq_dir,
    run_shell_cmd, find_longest_common_str
)

hiseq_type = '_r1'

# f = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/results/fruitfly_v1/CUT_and_RUN_Piwi_RNAi_embryo_3_4h_H3K4me3_rep2'
f = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/results/fruitfly_v1/CUT_and_RUN_White_RNAi_embryo_3_4h_H3K4me3_rep2'

x = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/results/fruitfly_v1/CUT_and_RUN_Piwi_RNAi_embryo_3_4h_H3K4me3_rep1'
x = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/results/fruitfly_v1/CUT_and_RUN_White_RNAi_embryo_3_4h_H3K4me3_rep1'

x = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/results/fruitfly_v1/CUT_and_RUN_White_RNAi_embryo_3_4h_H3K4me3'

# list_hiseq_file(x, 'align_json', 'r1')
a = read_hiseq(x, 'rn')
a.smp_name


In [ ]:
## test CnrR1

%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from hiseq.utils.file import list_file


data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'

fq1 = list_file(data_dir, '*Piwi*K4*1.fq.gz')
fq2 = list_file(data_dir, '*Piwi*K4*2.fq.gz')
outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

args = {
    'fq1': fq1[1],
    'fq2': fq2[1],
    'genome': 'dm6',
    'outdir': outdir,
    'threads': 12,
    'parallel_jobs': 2
}

CnrR1(**args).run()




In [ ]:
## test CnrRn

%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from cnr_rn import CnrRn
from hiseq.utils.file import list_file
from hiseq.utils.utils import read_hiseq, Config, list_hiseq_file
from utils import qc_frip

# data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'


# fq1 = list_file(data_dir, '*Piwi*IgG*1.fq.gz')
# fq2 = list_file(data_dir, '*Piwi*IgG*2.fq.gz')

# outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

# args = {
#     'fq1': fq1,
#     'fq2': fq2,
#     'genome': 'dm6',
#     'outdir': outdir,
#     'threads': 12,
#     'parallel_jobs': 4
# }

# CnrRn(**args).run()


c = '/data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG/config/config.yaml'
args = Config().load(c)
a = CnrRn(**args)
# a.run_pipe_r1()
a.run()

# rep_dir = '/data/yulab/wangming/work/yu_2021/projects/20210421_lxh_CnR/results/CnR_v2/CnR_Nos_DaGal4XshPiwi_6-8h_IgG_rep1'
# k = 'bam'
# list_hiseq_file(rep_dir, k, 'r1')



# x = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly/CUT_and_RUN_Piwi_RNAi_embryo_3_4h_H3K4me3/'
# pd = read_hiseq(x)
# qc_frip(x)

In [ ]:
# test CnrRx # design

%reload_ext autoreload
%autoreload 2
import os
import sys
from cnr_r1 import CnrR1
from cnr_rn import CnrRn
from cnr_rx import CnrRx
from hiseq.utils.file import list_file
from hiseq.utils.utils import read_hiseq, print_dict
from utils import qc_frip
data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'

ip_fq1 = list_file(data_dir, '*White*K4*1.fq.gz')
ip_fq2 = list_file(data_dir, '*White*K4*2.fq.gz')
input_fq1 = list_file(data_dir, '*White*IgG*1.fq.gz')
input_fq2 = list_file(data_dir, '*White*IgG*2.fq.gz')

outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

args = {
    'ip_fq1': ip_fq1,
    'ip_fq2': ip_fq2,
    'input_fq1': input_fq1,
    'input_fq2': input_fq2,
    'genome': 'dm6',
    'outdir': outdir,
    'threads': 12,
    'parallel_jobs': 4,
    'gene_bed': '/home/wangming/data/genome/dm6/annotation_and_repeats/dm6.ensembl.bed',
}

CnrRx(**args).run()


In [ ]:
import pandas as pd


def parse_report(in_report, out_stat, topN=10, tax_level='G'):
        """
        Parameters
        ----------
        in_report: str
            The --report output of kraken2
            
        out_stat: str
            The file saving the simplified report, by specific tax level
            
        topN: int
            The top N taxon, default: 100
            
        tax_level: str
            The specific tax level to show, default: ['G'],
            options: [D, K, P, C, O, F, G, S]
        
        parse Kraken2 report (--report)
        Inspect the output of kraken2 report file
        pandas table
        
        col-1: Percentage of fragments covered by the clade rooted at this taxon
        col-2: Number of fragments covered by the clade rooted at this taxon
        col-3: Number of fragments assigned directly to this taxon
        col-4: A rank code, indicating (U)nclassified, (R)oot, (D)omain, 
               (K)ingdom, (P)hylum, (C)lass, (O)rder, (F)amily, (G)enus,
               or (S)pecies. Taxa that are not at any of these 10 ranks have 
               a rank code that is formed by using the rank code of the closest
               ancestor rank with a number indicating the distance from that rank.
               E.g., "G2" is a rank code indicating a taxon is between genus and 
               species and the grandparent taxon is at the genus rank.
        col-5: NCBI taxonomic ID number
        col-6: Indented scientific name
        
        # choose the top species, by col-3, read directly in taxon
        """
        s = ['pct', 'reads_in_clade', 'reads_in_tax', 'code', 'taxid', 'name']
        try:
            df1 = pd.read_csv(in_report, '\t', names=s)
        except IOError as e:
            log.error(e)
            return None
        # choose Geneus (G); root, unclassified
        df_un = df1.loc[df1['code'] == 'U'] # unclassified
        df_root = df1.loc[df1['code'] == 'R'] # root
        df_tax = df1.loc[df1['code'].isin([tax_level]), ] # eg: G
        # choose top ranked taxon
        df_tax = df_tax.sort_values(['reads_in_tax'], ascending=False)
        # sub-sample
        df2 = df_tax.loc[0:topN,]
#         # combine
#         df = pd.concate([df_un, df2])
        ## remove white spaces
#         df['name'] = df['name'].str.strip()
#         # add sample name
#         df['sample'] = file_prefix(x) # prefix
#         # get total
#         n_hit = df_root.loc[:, 'reads_in_clade']
#         n_unhit = df_un.loc[:, 'reads_in_clade']
#         # adt pct
#         df['reads_hit'] = int(n_hit) # root
#         df['reads_total'] = int(n_hit) + int(n_unhit) # root + unclassified
#         df['hit_pct'] = df['reads_in_tax'] / int(n_hit) * 100
#          # sub-sample
#         df3 = df.loc[:,['sample', 'name', 'reads_in_tax', 'hit_pct', 'reads_hit', 'reads_total']]        
#         # custome options
#         with pd.option_context('display.expand_frame_repr', False, 'display.max_colwidth', 20):
#             print(df3)
#         # save to file
#         df3.to_csv(out_stat, sep='\t', index=False)
#         return df3

in_report = '/data/yulab/wangming/work/devel_pipeline/hiseq/qc/aaaa.kraken2.report'
out_stat = '/data/yulab/wangming/work/devel_pipeline/hiseq/qc/aaaa.kraken2.stat'
topN = 10
# parse_report(in_report, out_stat)
tax_level = 'G'
s = ['pct', 'reads_in_clade', 'reads_in_tax', 'code', 'taxid', 'name']
try:
    df1 = pd.read_csv(in_report, '\t', names=s)
except IOError as e:
    log.error(e)
# choose Geneus (G); root, unclassified
df_un = df1.loc[df1['code'] == 'U'] # unclassified
df_root = df1.loc[df1['code'] == 'R'] # root
df_tax = df1.loc[df1['code'].isin([tax_level]), ] # eg: G
# choose top ranked taxon
df_tax = df_tax.sort_values(['reads_in_tax'], ascending=False)
# sub-sample
df2 = df_tax.iloc[0:topN,]
df2

In [ ]:
 all([False])

In [ ]:
# test CnrRd # design
%reload_ext autoreload
%autoreload 2
from cnr_rd import CnrRd

data_dir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/data/fruitfly/'

ip_fq1 = list_file(data_dir, '*Piwi*K4*1.fq.gz')
ip_fq2 = list_file(data_dir, '*Piwi*K4*2.fq.gz')

input_fq1 = list_file(data_dir, '*Piwi*IgG*1.fq.gz')
input_fq2 = list_file(data_dir, '*Piwi*IgG*2.fq.gz')
outdir = '/data/yulab/wangming/work/devel_pipeline/hiseq/cnr/output/fruitfly'

args = {
    'design': 'aaa.toml',
    'fq_dir': data_dir,
    'ip_fq1': ip_fq1,
    'ip_fq2': ip_fq2,
    'input_fq1': input_fq1,
    'input_fq2': input_fq2
}


args = {
    'design': 'aaa.toml',
    'fq_dir': data_dir,
    'ip': ['Piwi_RNAi_embryo_3_4h_H3K4me3', 'White_RNAi_embryo_3_4h_H3K4me3'],
    'input': ['Piwi_RNAi_embryo_3_4h_IgG'],
}

a = CnrRd(**args)

#a.check_fx_args()
df = a.run()

In [ ]:

from difflib import SequenceMatcher

string1 = "apple pie available"
string2 = "come have some apple pies"

# match = SequenceMatcher(None, string1, string2).find_longest_match(0, len(string1), 0, len(string2))

# print(match)  # -> Match(a=0, b=15, size=9)
# print(string1[match.a: match.a + match.size])  # -> apple pie
# print(string2[match.b: match.b + match.size])  # -> apple pie

# string1[match.a:match.a+match.size]

# dir(match)


## Temp
def find_longest_common_str(s1, s2):
    if isinstance(s1, str) and isinstance(s2, str):
        m = SequenceMatcher(None, s1, s2) # match
        l = m.find_longest_match(0, len(s1), 0, len(s2))
        out = s1[l.a: l.a + l.size]
    else:
        log.error('only support str, got s1={} s2={}'.type(
            type(s1).__name__, type(s2).__name__))
        out = None
    return out

find_longest_common_str('abc', 'efgab')


In [3]:
%reload_ext autoreload
%autoreload 2

import os
from hiseq.utils.utils import list_hiseq_file
from hiseq.utils.file import copy_file
from cnr_merge import *


args = {
    'indir': '/data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT',
    'outdir': '/data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/merge',
}

a = CnrMerge(**args)
a.run()
# a.project_dir
# copy_hiseq_qc(a.project_dir)


[2021-10-22 16:50:42 WARNING] unknown hiseq dir: /data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT
[2021-10-22 16:50:42 WARNING] copy_file() failed, src not vaild: /data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT/CnT_mHaploid_IgG/qc/00.trim_summary.json
[2021-10-22 16:50:42 WARNING] copy_file() failed, src not vaild: /data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT/CnT_mHaploid_IgG/qc/01.alignment_summary.json
[2021-10-22 16:50:42 WARNING] copy_file() failed, src not vaild: /data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT/CnT_mHaploid_IgG/qc/01.pcr_dup_summary.json
[2021-10-22 16:50:42 WARNING] copy_file() failed, src not vaild: /data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT/CnT_mHaploid_IgG/qc/02.length_distribution.csv
[2021-10-22 16:50:42 WARNING] copy_file() failed, src not vaild: /data/yulab/wangming/work/yu_2021/proj

In [8]:
import hashlib
import uuid
## for hash string  

## for hash string; version with random number
def hash_string(s):
    return hashlib.sha256(s.encode()).hexdigest()


def check_hash_string(hash_s, s):
    """
    Parameters:
    hash_s  : str
        The SHA-256 value for the input string 
        also, could be the first few characters of the SHA-256 value
        
    s  : str
        The string for checking
    """
    # hash_s, could be the first few characters, (8?)
    hs = hash_string(s)
    return hs.startswith(hash_s) or hs == s

# a = hash_string('hello') 
# # check_hashed_string(a, 'hello')
# b = '2cf24dba'

# check_hash_string(b, 'hello')
# len(a)

s = '/data/yulab/wangming/work/yu_2021/projects/20211021_qd_CnT_mouse_yy80/results/CnT'

hash_string(s)

# s.encode()

# hashlib.sha256(s).hexdigest()


'2fc06e87f49f8993d32004be09beb02799f5bfa5ccc0c7eb71a99d71fe4675a6'